In [ ]:
import os

In [ ]:
os.chdir(os.path.split(os.getcwd())[0])

In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
import gym
from agent import *
from optionpricing import *
import yaml
import torch
from collections import defaultdict

In [ ]:
import matplotlib.style as style

In [ ]:
style.use('seaborn-poster')

In [ ]:
experiment_folder = None

In [ ]:
with open(os.path.join('experiments', experiment_folder, 'config.yaml'), 'r') as f:
    args_dict = yaml.load(f, Loader = yaml.SafeLoader)

In [ ]:
class Args:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

In [ ]:
args = Args(**args_dict)

In [ ]:
config = {
        'S': 100,
        'T': 10, # 10 days
        'L': 1,
        'm': 100, # L options for m stocks
        'n': 0,
        'K': [95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105],
        'D': 5,
        'mu': 0,
        'sigma': 0.01,
        'r': 0,
        'ss': 5,
        'kappa': 0.1,
        'multiplier': args.trc_multiplier,
        'ticksize': args.trc_ticksize
        }

env = OptionPricingEnv(config)
env.configure()

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def load_estimator(env, device, nhidden, nunits, experiment_folder, kind = 'best'):
    state_shape = env.observation_space.shape
    state_space_dim = state_shape[0] if len(state_shape) == 1 else state_shape
    
    estimator = Estimator(nhidden, nunits, state_space_dim, env.action_space.n)
    if kind == 'best':
        checkpoint = torch.load(os.path.join('experiments', experiment_folder, 'best.pt'), map_location = torch.device('cpu'))
    elif kind == 'checkpoint':
        checkpoint = torch.load(os.path.join('experiments', experiment_folder, 'checkpoint.pt'), map_location = torch.device('cpu'))
    else:
        raise ValueError('Invalid choice for kind')
        
    estimator.load_state_dict(checkpoint['estimator'])
    estimator.eval()
    
    return estimator

In [ ]:
def delta_neutral_policy(env):
    return env.inv_action_map[-1 * int(env.delta * (env.L * env.m)) - env.n]

In [ ]:
def simulate_episodes(config, n = 10000, kind = 'agent'):
    env = OptionPricingEnv(config)
    env.configure()
    estimator = load_estimator(env, device, args.nhidden, args.nunits, experiment_folder, 'best')
    
    full_history = {}
    for i in range(1, n + 1): 
        print(f'\r{i}/{n} | {100 * i / n:.2f} %', end = '', flush = True)
        state = torch.from_numpy(env.reset()).to(device)
        history = defaultdict(list)
        done = False

        while not done:
            history['delta'].append(env.delta)
            if kind == 'agent':
                with torch.no_grad():
                    action = np.argmax(estimator(state).numpy())
            else:
                action = delta_neutral_policy(env)
                
            state, reward, done, info = env.step(action)

            history['reward'].append(reward)
            history['n'].append(env.n)
            history['stock_value'].append(env.stock_value)
            history['option_value'].append(env.option_value)
            history['cash'].append(env.cash)
            history['cost'].append(info['cost'])
            history['pnl'].append(info['pnl'])
            state = torch.from_numpy(state).to(device)
        full_history[i] = history

    return full_history

In [ ]:
from scipy.stats import gaussian_kde

### K = 100

In [ ]:
config['K'] = 100
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
agent_history = simulate_episodes(config, n = 1000)
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
delta_history = simulate_episodes(config, n = 1000, kind = 'delta')

In [ ]:
agent_pnl_volatility = [np.std(agent_history[i]['pnl']) for i in range(1, len(agent_history) + 1)]
delta_pnl_volatility = [np.std(delta_history[i]['pnl']) for i in range(1, len(delta_history) + 1)]
agent_costs = [sum(agent_history[i]['cost']) for i in range(1, len(agent_history) + 1)]
delta_costs = [sum(delta_history[i]['cost']) for i in range(1, len(delta_history) + 1)]

In [ ]:
fig, ax = plt.subplots(figsize = (12, 18), nrows = 2, ncols = 1)
x_pnl_volatility = np.linspace(0, 25, 1000)
x_costs = np.linspace(0, 300, 1000)
delta_pnl_volatility_kernel = gaussian_kde(delta_pnl_volatility)
agent_pnl_volatility_kernel = gaussian_kde(agent_pnl_volatility)
delta_costs_kernel = gaussian_kde(delta_costs)
agent_costs_kernel = gaussian_kde(agent_costs)
y_delta_pnl_volatility = delta_pnl_volatility_kernel(x_pnl_volatility)
y_agent_pnl_volatility = agent_pnl_volatility_kernel(x_pnl_volatility)
y_delta_costs = delta_costs_kernel(x_costs)
y_agent_costs = agent_costs_kernel(x_costs)
ax[0].plot(x_pnl_volatility, y_delta_pnl_volatility, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_pnl_volatility):.2f}, Std: {np.std(delta_pnl_volatility):.2f}', color = 'blue')
ax[0].plot(x_pnl_volatility, y_agent_pnl_volatility, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_pnl_volatility):.2f}, Std: {np.std(agent_pnl_volatility):.2f}', color = 'red')
#ax[0].set_title('Volatility')
ax[0].set_xlabel('Volatility')
ax[0].set_ylabel('Density')
ax[0].legend()
ax[1].plot(x_costs, y_delta_costs, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_costs):.2f}, Std: {np.std(delta_costs):.2f}', color = 'blue')
ax[1].plot(x_costs, y_agent_costs, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_costs):.2f}, Std: {np.std(agent_costs):.2f}', color = 'red')
#ax[1].set_title('Costs')
ax[1].set_xlabel('Cost')
ax[1].set_ylabel('Density')
ax[1].legend()
plt.show()

### K = 95

In [ ]:
config['K'] = 95
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
agent_history = simulate_episodes(config, n = 1000)
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
delta_history = simulate_episodes(config, n = 1000, kind = 'delta')

In [ ]:
agent_pnl_volatility = [np.std(agent_history[i]['pnl']) for i in range(1, len(agent_history) + 1)]
delta_pnl_volatility = [np.std(delta_history[i]['pnl']) for i in range(1, len(delta_history) + 1)]
agent_costs = [sum(agent_history[i]['cost']) for i in range(1, len(agent_history) + 1)]
delta_costs = [sum(delta_history[i]['cost']) for i in range(1, len(delta_history) + 1)]

In [ ]:
fig, ax = plt.subplots(figsize = (12, 18), nrows = 2, ncols = 1)
x_pnl_volatility = np.linspace(0, 25, 1000)
x_costs = np.linspace(0, 300, 1000)
delta_pnl_volatility_kernel = gaussian_kde(delta_pnl_volatility)
agent_pnl_volatility_kernel = gaussian_kde(agent_pnl_volatility)
delta_costs_kernel = gaussian_kde(delta_costs)
agent_costs_kernel = gaussian_kde(agent_costs)
y_delta_pnl_volatility = delta_pnl_volatility_kernel(x_pnl_volatility)
y_agent_pnl_volatility = agent_pnl_volatility_kernel(x_pnl_volatility)
y_delta_costs = delta_costs_kernel(x_costs)
y_agent_costs = agent_costs_kernel(x_costs)
ax[0].plot(x_pnl_volatility, y_delta_pnl_volatility, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_pnl_volatility):.2f}, Std: {np.std(delta_pnl_volatility):.2f}', color = 'blue')
ax[0].plot(x_pnl_volatility, y_agent_pnl_volatility, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_pnl_volatility):.2f}, Std: {np.std(agent_pnl_volatility):.2f}', color = 'red')
#ax[0].set_title('Volatility')
ax[0].set_xlabel('Volatility')
ax[0].set_ylabel('Density')
ax[0].legend()
ax[1].plot(x_costs, y_delta_costs, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_costs):.2f}, Std: {np.std(delta_costs):.2f}', color = 'blue')
ax[1].plot(x_costs, y_agent_costs, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_costs):.2f}, Std: {np.std(agent_costs):.2f}', color = 'red')
#ax[1].set_title('Costs')
ax[1].set_xlabel('Cost')
ax[1].set_ylabel('Density')
ax[1].legend()
plt.show()

### K = 105

In [ ]:
config['K'] = 105
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
agent_history = simulate_episodes(config, n = 1000)
random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
delta_history = simulate_episodes(config, n = 1000, kind = 'delta')

In [ ]:
agent_pnl_volatility = [np.std(agent_history[i]['pnl']) for i in range(1, len(agent_history) + 1)]
delta_pnl_volatility = [np.std(delta_history[i]['pnl']) for i in range(1, len(delta_history) + 1)]
agent_costs = [sum(agent_history[i]['cost']) for i in range(1, len(agent_history) + 1)]
delta_costs = [sum(delta_history[i]['cost']) for i in range(1, len(delta_history) + 1)]

In [ ]:
fig, ax = plt.subplots(figsize = (12, 18), nrows = 2, ncols = 1)
x_pnl_volatility = np.linspace(0, 25, 1000)
x_costs = np.linspace(0, 300, 1000)
delta_pnl_volatility_kernel = gaussian_kde(delta_pnl_volatility)
agent_pnl_volatility_kernel = gaussian_kde(agent_pnl_volatility)
delta_costs_kernel = gaussian_kde(delta_costs)
agent_costs_kernel = gaussian_kde(agent_costs)
y_delta_pnl_volatility = delta_pnl_volatility_kernel(x_pnl_volatility)
y_agent_pnl_volatility = agent_pnl_volatility_kernel(x_pnl_volatility)
y_delta_costs = delta_costs_kernel(x_costs)
y_agent_costs = agent_costs_kernel(x_costs)
ax[0].plot(x_pnl_volatility, y_delta_pnl_volatility, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_pnl_volatility):.2f}, Std: {np.std(delta_pnl_volatility):.2f}', color = 'blue')
ax[0].plot(x_pnl_volatility, y_agent_pnl_volatility, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_pnl_volatility):.2f}, Std: {np.std(agent_pnl_volatility):.2f}', color = 'red')
#ax[0].set_title('Volatility')
ax[0].set_xlabel('Volatility')
ax[0].set_ylabel('Density')
ax[0].legend()
ax[1].plot(x_costs, y_delta_costs, lw = 1.5, label = f'Delta | Mean: {np.mean(delta_costs):.2f}, Std: {np.std(delta_costs):.2f}', color = 'blue')
ax[1].plot(x_costs, y_agent_costs, lw = 1.5, label = f'Agent | Mean: {np.mean(agent_costs):.2f}, Std: {np.std(agent_costs):.2f}', color = 'red')
#ax[1].set_title('Costs')
ax[1].set_xlabel('Cost')
ax[1].set_ylabel('Density')
ax[1].legend()
plt.show()